In [2]:
import netCDF4
import pyproj
import numpy as np
import pandas as pd

In [3]:
ds = netCDF4.Dataset(r'C:\Users\vries_cy\OneDrive - Stichting Deltares\Documents\My Received Files\SDN_2015-09_TS_MedSea_QC_done_v2_130ef42_1\SDN_2015-09_TS_MedSea_QC_done_v2_130ef42_1.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    comment: ODV NetCDF Export File V2.0
    Creator: root@525bb4627be0
    CreateTime: 2020-01-16T08:50:44
    Software: Web Ocean Data View 5.1.8 - web_server: 14 - 64 bit (Linux)
    source: /var/www/html/ODV_Data/SDN_2015-09_TS_MedSea_QC_done_v2.odv
    SourceLastModified: 2015-07-21T11:19:34
    DataField: Ocean
    DataType: Profiles
    featureType: profile
    dimensions(sizes): N_STATIONS(4608), N_SAMPLES(14175), STRING34(34), STRING23(23)
    variables(dimensions): |S1 metavar1(N_STATIONS,STRING23), |S1 metavar2(N_STATIONS,STRING34), |S1 metavar3(N_STATIONS), float32 longitude(N_STATIONS), float32 latitude(N_STATIONS), float64 date_time(N_STATIONS), float32 var1(N_STATIONS,N_SAMPLES), int8 var1_qc(N_STATIONS,N_SAMPLES), float32 var2(N_STATIONS,N_SAMPLES), int8 var2_qc(N_STATIONS,N_SAMPLES), float32 var3(N_STATIONS,N_SAMPLES), int8 var3_qc(N_STATIONS,N_SAMPLES)
    gro

In [4]:
station_lat = ds.variables['latitude'][:]
station_lon = ds.variables['longitude'][:]
lon_i = 10
lat_i = 30
lon = np.zeros_like(station_lon) + lon_i
lat = np.zeros_like(station_lat) + lat_i

In [5]:
wgs84 = pyproj.Geod(ellps='WGS84')


In [6]:
_, _, distance = wgs84.inv(
    lon, lat,
    station_lon, station_lat
)


In [7]:
idx = distance.argmin()
idx

2266

In [8]:
var = ds.variables['metavar1']
var.dtype == 'S1'

True

In [13]:
cruise = netCDF4.chartostring(ds.variables['metavar1'][idx])
profile = netCDF4.chartostring(ds.variables['metavar2'][idx])
type_ = ds.variables['metavar3'][idx]
# cdi_id = netCDF4.chartostring(ds.variables['metavar4'][idx])
# edmo_code = ds.variables['metavar5'][idx]
profile

array('1900602_35_A', dtype='<U34')

In [15]:
# cdi_ids = netCDF4.chartostring(ds.variables['metavar4'][:])

In [18]:
# %%timeit
# np.argmax(cdi_ids == 'FS35196900000_90460_H13')

In [19]:
var_names = [
    name 
    for name, var 
    in ds.variables.items() 
    if name.startswith('var') and not '_' in name
]

# add the variables to the list
variables = {}
for var_name in var_names:
    var = ds.variables[var_name]
    variables[var.long_name] = var[idx]

df = pd.DataFrame(data=variables)
# get rid of missing data
df = df.dropna(how='all')

# get metadata
date_nums = ds.variables['date_time'][idx]
date_units = ds.variables['date_time'].units
date = netCDF4.num2date(date_nums, date_units)
records = df.to_json(orient='records')
response = {
    "series": records,
    "meta": {
        "date": date
    }
}
response

{'series': '[{"Depth":0.0,"ITS-90 water temperature":16.8509998322,"Water body salinity":38.0719985962},{"Depth":15.8850002289,"ITS-90 water temperature":16.8439998627,"Water body salinity":38.0769996643},{"Depth":35.7400016785,"ITS-90 water temperature":16.8449993134,"Water body salinity":38.0769996643},{"Depth":55.591999054,"ITS-90 water temperature":16.8409996033,"Water body salinity":38.0830001831},{"Depth":65.5179977417,"ITS-90 water temperature":16.892999649,"Water body salinity":38.1759986877},{"Depth":75.4430007935,"ITS-90 water temperature":17.1100006104,"Water body salinity":38.2869987488},{"Depth":84.3759994507,"ITS-90 water temperature":17.2490005493,"Water body salinity":38.3979988098},{"Depth":94.3000030518,"ITS-90 water temperature":17.281999588,"Water body salinity":38.4150009155},{"Depth":105.216003418,"ITS-90 water temperature":17.1989994049,"Water body salinity":38.4119987488},{"Depth":115.138999939,"ITS-90 water temperature":17.1849994659,"Water body salinity":38.42

In [20]:
date_nums.shape
ds.variables['var2'].shape

(4608, 14175)

In [21]:
ds.variables['var1']

<class 'netCDF4._netCDF4.Variable'>
float32 var1(N_STATIONS, N_SAMPLES)
    positive: down
    long_name: Depth
    units: m
    comment: Codes: SDN:P01::ADEPZZ01 SDN:P06::ULAA
    ancillary_variables: var1_qc
    C_format: %.1f
    FORTRAN_format: F12.1
    _FillValue: -10000000000.0
unlimited dimensions: 
current shape = (4608, 14175)
filling on